In [ ]:
#use python to get core as defined by roary and then cross with metaGs
- Used L. amylovorus but this isn't present in humans
- Different species that is present in both?


In [ ]:
/group/ctbrowngrp/sourmash-db/gtdb-rs220/gtdb-rs220.lineages.csv
/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/MAGs/taxonomy_MAGs.forsmash.csv

# Gemmiger qucibialis

In [ ]:
# get lists of genes
python /group/ctbrowngrp2/amhorst/2025-pangenome/workflow/scripts/count_genes.py \
gene_presence_absence.Rtab

# clean fasta file headers
mamba activate seqtk
seqtk seq -A pan_genome_reference.fa | sed 's/^>\([^ ]*\) \(.*\)/>\2/' > pan_genome_reference.clean.fa

# get cloud/core from fasta
mamba activate bbmap
filterbyname.sh in=pan_genome_reference.clean.fa \
out=cloud.fa names=cloud.txt include=t substring=f

filterbyname.sh in=pan_genome_reference.clean.fa \
out=hard_core.fa names=hard_core.txt include=t substring=f

In [ ]:
# sketch and pangenome it
mamba activate branchwater-skipmer
sourmash sketch dna cloud.fa -o cloud.zip -p k=21,scaled=100 

# try pangenome merge and ranktable (m2n3, k21, s50)
sourmash scripts pangenome_merge cloud.zip -k 21  \
-o cloud.pang.zip --scaled 100 

sourmash scripts pangenome_ranktable \
cloud.pang.zip -o cloud.rank.csv \
-k 21 --scaled 100 

sourmash sketch dna hard_core.fa -o hard_core.zip -p k=21,scaled=100 
# try pangenome merge and ranktable (m2n3, k21, s50)
sourmash scripts pangenome_merge hard_core.zip -k 21  \
-o hard_core.pang.zip --scaled 100 
sourmash scripts pangenome_ranktable \
hard_core.pang.zip -o hard_core.rank.csv \
-k 21 --scaled 100 

In [ ]:
# snakemake x metags
srun --account=ctbrowngrp -p med2 -J roary -t 1:00:00 -c 10 --mem=50gb --pty bash

snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 3 --rerun-incomplete -k -s metag_pang.smk -n

In [ ]:
## G Quci
# 130 ref genomes
# 44 new genomes
roary -p 15 -f roary_gtdb \
-e -n -v gff_gtdb/*.gff 


roary -p 18 -f roary_pig \
-e -n -v gff_pig/*.gff 